In [105]:
import pandas as pd
import numpy as np

# <b>NOTES<b>

## <b>Ideas:<b>
    -Merge Wave1, Wave2 and Wave3 in order to look at the same population throughout and check the differences
    -
    -
    -

    
    
    

In [110]:
#Convert from .dta to .csv
df = pd.read_stata("Wave2.dta")
# df.to_csv('Wave2.csv')

# df = pd.read_csv("Wave2.csv", index_col="pid", delimiter=",", low_memory=False)
display(df.head())
df.info()

,pid,w2_nc_outcome,w2_nc_intrv_c,w2_nc_intrv_d,w2_nc_intrv_m,w2_nc_intrv_y,w2_nc_duration,w2_nc_refexpl,w2_nc_refexpl_o,w2_nc_dob_m,...,w2_nc_cvnomask3,w2_nc_cvnomask_o,w2_nc_cvalcohol,w2_nc_cvstayhome,w2_nc_mnthlth_1,w2_nc_mnthlth_2,w2_nc_intres,w2_nc_intlng1,w2_nc_intlng2,w2_nc_intlng3
0,301013,Not Contacted in Wave 2,Missing,-3,NaN,-3,0.0,NaN,,NaN,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,301058,Not Contacted in Wave 2,660,3rd,August,2020,0.0,NaN,,NaN,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,301059,Refused/ Not Available,626,7th,August,2020,0.0,Permanent refusal,,NaN,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,301062,Not Contacted in Wave 2,631,8th,August,2020,0.0,NaN,,NaN,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,301067,Successfully Interviewed,621,18th,July,2020,17.08,NaN,,July,...,NaN,,Most people,Most people,Not at all,Not at all,Friendly,Sepedi,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7051 entries, 0 to 7050
Columns: 177 entries, pid to w2_nc_intlng3
dtypes: category(163), int32(1), object(13)
memory usage: 2.0+ MB


In [111]:
df = df[df['w2_nc_outcome'] == "Successfully Interviewed"]
display(df.head())
df.info()

,pid,w2_nc_outcome,w2_nc_intrv_c,w2_nc_intrv_d,w2_nc_intrv_m,w2_nc_intrv_y,w2_nc_duration,w2_nc_refexpl,w2_nc_refexpl_o,w2_nc_dob_m,...,w2_nc_cvnomask3,w2_nc_cvnomask_o,w2_nc_cvalcohol,w2_nc_cvstayhome,w2_nc_mnthlth_1,w2_nc_mnthlth_2,w2_nc_intres,w2_nc_intlng1,w2_nc_intlng2,w2_nc_intlng3
4,301067,Successfully Interviewed,621,18th,July,2020,17.08,NaN,,July,...,NaN,,Most people,Most people,Not at all,Not at all,Friendly,Sepedi,NaN,NaN
5,301070,Successfully Interviewed,644,28th,July,2020,14.67,NaN,,January,...,NaN,,Most people,A few people,Not at all,Not at all,Friendly,IsiZulu,NaN,NaN
6,301073,Successfully Interviewed,606,21st,July,2020,27.17,NaN,,July,...,NaN,,Most people,A few people,More than half the days,Several days,Friendly,Sesotho,NaN,NaN
7,301079,Successfully Interviewed,682,17th,July,2020,11.68,NaN,,February,...,NaN,,None,Everyone stayed home,Not at all,Several days,Neither hostile nor friendly,Sesotho,NaN,NaN
8,301081,Successfully Interviewed,634,20th,July,2020,23.72,NaN,,Don't know,...,NaN,,A few people,Most people,Not at all,Not at all,Friendly,IsiNdebele,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5676 entries, 4 to 7050
Columns: 177 entries, pid to w2_nc_intlng3
dtypes: category(163), int32(1), object(13)
memory usage: 1.7+ MB


We have now narrrowed down our dataset to only the people who were successfully interviewed.